Going to play with different Neural Networks to try to model all of the data. I don't seem to have saved a varient of the dataframe where all of the stations are present

In [1]:
import polars as pl
from tqdm import tqdm
import os
import pickle

In [2]:
# use CSVs in dir
# file_path = f"data/tfl_CSVs/"
directory = f"data/tfl_CSVs/"
downloaded_files = set(os.listdir(directory))
CSVs = downloaded_files
print(f"There are {len(CSVs)} files: {CSVs}")

There are 417 files: {'229JourneyDataExtract26Aug2020-01Sep2020.csv', '344JourneyDataExtract14Nov2022-20Nov2022.csv', '316JourneyDataExtract04May2022-10May2022.csv', '308JourneyDataExtract09Mar2022-15Mar2022.csv', '317JourneyDataExtract11May2022-17May2022.csv', '65JourneyDataExtract05Jul2017-11Jul2017.csv', '333JourneyDataExtract31Aug2022-06Sep2022.csv', '120JourneyDataExtract25July2018-31July2018.csv', '104JourneyDataExtract04Apr2018-10Apr2018.csv', '91JourneyDataExtract03Jan2018-09Jan2018.csv', '145JourneyDataExtract16Jan2019-22Jan2019.csv', '26JourneyDataExtract05Oct2016-11Oct2016.csv', '94JourneyDataExtract24Jan2018-30Jan2018.csv', '108JourneyDataExtract02May2018-08May2018.csv', '04JourneyDataExtract01Apr2016-30Apr2016.csv', '268JourneyDataExtract02Jun2021-08Jun2021.csv', '16JourneyDataExtract27Jul2016-02Aug2016.csv', '345JourneyDataExtract21Nov2022-27Nov2022.csv', '197JourneyDataExtract15Jan2020-21Jan2020.csv', '306JourneyDataExtract23Feb2022-01Mar2022.csv', '346JourneyDataExtract

In [3]:
# CSVs = CSVs[:50]

In [ ]:
# Initialize an empty Polars DataFrame

CSVs_to_df = True
if CSVs_to_df:
    merged_df = pl.DataFrame()
    cols = []

    progress_bar = tqdm(CSVs, desc="Processing")

    for idx, csv in enumerate(progress_bar):
        file_path = f"{directory}{csv}"
        # print(f"CSV: {csv}\n file_path: {file_path}")
        df = pl.read_csv(file_path, skip_rows_after_header=1, ignore_errors=True)
        # print(df.columns)

        # Rename columns and keep only 'Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name'
        renaming_dict = {
            'Number': 'Rental ID',
            'rental ID': 'Rental ID',
            'Rental Id': 'Rental ID',
            'Start date': 'Start Date',
            'StartStation Name': 'Start Station Name',
            'Start station': 'Start Station Name',
            'End date': 'End Date',
            'EndStation Name': 'End Station Name',
            'End station': 'End Station Name'
        }

        for old_col, new_col in renaming_dict.items():
            if old_col in df.columns:
                df = df.rename({old_col: new_col})
        
        cols_to_keep = ['Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name']
        df = df.select(cols_to_keep)

        # Convert date columns
        df = df.with_columns([
            pl.col('Start Date').str.strptime(pl.Datetime, strict=False),
            pl.col('End Date').str.strptime(pl.Datetime, strict=False),
            pl.col('Start Station Name').cast(pl.Utf8),
            pl.col('End Station Name').cast(pl.Utf8)
        ])

        # Filter rows (if needed)
        # df = df.filter((pl.col("Start Station Name") == "Waterloo Station 3, Waterloo") |
        #                (pl.col("End Station Name") == "Waterloo Station 3, Waterloo"))
        
        # Concatenate DataFrames
        merged_df = pl.concat([merged_df, df])

        progress_bar.set_description(
            f"Processing Index: {idx+1}/{len(CSVs)}, merged_df is {merged_df.estimated_size()/(1024*1024)}MB large"
        )

Processing Index: 396/417, merged_df is 6610.522654533386MB large:  95%|█████████▍| 396/417 [00:59<00:04,  4.31it/s] 

In [ ]:
pickle_dir = "pickle/CSV_data_df"
if CSVs_to_df:
    # save to pickle
    with open(pickle_dir, 'wb') as f:
        pickle.dump(merged_df, f)
else:
    #load from pickle
    with open(pickle_dir, 'rb') as f:
        df = pickle.load(f)

df.head()

: 

In [3]:
start_stations = df.select(pl.col('Start Station Name').alias('Station Name'))
end_stations = df.select(pl.col('End Station Name').alias('Station Name'))

unique_values = (
    pl.concat([start_stations, end_stations])
    .unique()
    .sort("Station Name")
)
id_mapping = unique_values.with_row_count("ID")
print(id_mapping)

shape: (786, 2)
┌─────┬─────────────────────────────────┐
│ ID  ┆ Station Name                    │
│ --- ┆ ---                             │
│ u32 ┆ str                             │
╞═════╪═════════════════════════════════╡
│ 0   ┆ Abbey Orchard Street, Westmins… │
│ 1   ┆ Abbotsbury Road, Holland Park   │
│ 2   ┆ Aberdeen Place, St. John's Woo… │
│ 3   ┆ Aberfeldy Street, Poplar        │
│ 4   ┆ Abingdon Green, Westminster     │
│ …   ┆ …                               │
│ 781 ┆ Wormwood Street, Liverpool Str… │
│ 782 ┆ Wren Street, Holborn            │
│ 783 ┆ Wright's Lane, Kensington       │
│ 784 ┆ Wynne Road, Stockwell           │
│ 785 ┆ York Hall, Bethnal Green        │
└─────┴─────────────────────────────────┘


/tmp/ipykernel_6504/363409236.py:9: DeprecationWarning: `DataFrame.with_row_count` is deprecated. Use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  id_mapping = unique_values.with_row_count("ID")


In [4]:
df_with_ids = df.join(id_mapping.rename({"Station Name": "Start Station Name", "ID": "Start_ID"}), on="Start Station Name", how="left")
df_with_ids = df_with_ids.join(id_mapping.rename({"Station Name": "End Station Name", "ID": "End_ID"}), on="End Station Name", how="left")

df_with_ids.head()

Rental ID,Start Date,Start Station Name,End Date,End Station Name,Start_ID,End_ID
i64,datetime[μs],str,datetime[μs],str,u32,u32
74656682,2018-04-18 07:32:00,"""Crosswall, Tower""",2018-04-18 07:43:00,"""Tallis Street, Temple""",182,684
74699122,2018-04-19 07:31:00,"""Crosswall, Tower""",2018-04-19 07:42:00,"""Tallis Street, Temple""",182,684
74831922,2018-04-21 22:42:00,"""Moor Street, Soho""",2018-04-21 23:14:00,"""Geraldine Street, Elephant & C…",452,268
74740907,2018-04-19 22:51:00,"""Moor Street, Soho""",2018-04-19 23:10:00,"""Walworth Road, Elephant & Cast…",452,731
74787286,2018-04-20 22:31:00,"""Moor Street, Soho""",2018-04-20 22:46:00,"""Ontario Street, Elephant & Cas…",452,496


In [5]:
df_brief = df_with_ids["Rental ID", "Start Date", "End Date", "Start_ID", "End_ID"]
df_brief.head()

Rental ID,Start Date,End Date,Start_ID,End_ID
i64,datetime[μs],datetime[μs],u32,u32
74656682,2018-04-18 07:32:00,2018-04-18 07:43:00,182,684
74699122,2018-04-19 07:31:00,2018-04-19 07:42:00,182,684
74831922,2018-04-21 22:42:00,2018-04-21 23:14:00,452,268
74740907,2018-04-19 22:51:00,2018-04-19 23:10:00,452,731
74787286,2018-04-20 22:31:00,2018-04-20 22:46:00,452,496


In [8]:
df_start = df_brief["Rental ID", "Start Date", "Start_ID"].rename({"Start Date": "Date", "Start_ID": "ID"}).with_columns(pl.lit(-1).alias("flow"))

df_end = df_brief["Rental ID", "End Date", "End_ID"].rename({"End Date": "Date", "End_ID": "ID"}).with_columns(pl.lit(1).alias("flow"))

df_flows = pl.concat([df_start, df_end]).sort("Date")
df_flows.head(50)

Rental ID,Date,ID,flow
i64,datetime[μs],u32,i32
74654037,2018-04-18 00:00:00,551,-1
74654041,2018-04-18 00:00:00,204,-1
74654038,2018-04-18 00:00:00,552,-1
74654036,2018-04-18 00:00:00,670,-1
74654039,2018-04-18 00:00:00,695,-1
…,…,…,…
74654057,2018-04-18 00:10:00,587,1
74654066,2018-04-18 00:10:00,361,1
74654049,2018-04-18 00:10:00,83,1


In [9]:
df_flows_id = df_flows.group_by("ID")
df_flows_id.head()

ID,Rental ID,Date,flow
u32,i64,datetime[μs],i32
304,74654047,2018-04-18 00:01:00,-1
304,74654403,2018-04-18 03:47:00,-1
304,74654707,2018-04-18 06:19:00,-1
304,74655873,2018-04-18 07:12:00,-1
304,74656789,2018-04-18 07:35:00,-1
…,…,…,…
512,74656298,2018-04-18 07:24:00,-1
512,74657379,2018-04-18 07:45:00,-1
512,74657476,2018-04-18 07:47:00,-1


In [17]:
df_flows.describe()

statistic,Rental ID,Date,ID,flow
str,f64,str,f64,f64
"""count""",551084.0,"""551084""",551084.0,551084.0
"""null_count""",0.0,"""0""",0.0,0.0
"""mean""",7.4800e7,"""2018-04-21 10:54:29.198126""",392.997998,0.0
"""std""",84004.495487,null,234.76011,1.000001
"""min""",7.4654033e7,"""2018-04-18 00:00:00""",0.0,-1.0
"""25%""",7.4726946e7,"""2018-04-19 18:01:00""",189.0,-1.0
"""50%""",7.4799651e7,"""2018-04-21 12:24:00""",388.0,1.0
"""75%""",7.4872807e7,"""2018-04-22 23:10:00""",596.0,1.0
"""max""",7.4944543e7,"""2018-04-24 23:59:00""",785.0,1.0


In [16]:
df_summed_by_day = (
    df_flows
    .with_columns(pl.col("Date").dt.truncate("1d").alias("day"))
    .group_by(["ID", "day"])
    .agg(pl.col("flow").sum().alias("total_flow"))
)

print(df_summed_by_day)

shape: (5_475, 3)
┌─────┬─────────────────────┬────────────┐
│ ID  ┆ day                 ┆ total_flow │
│ --- ┆ ---                 ┆ ---        │
│ u32 ┆ datetime[μs]        ┆ i32        │
╞═════╪═════════════════════╪════════════╡
│ 161 ┆ 2018-04-24 00:00:00 ┆ -1         │
│ 248 ┆ 2018-04-18 00:00:00 ┆ -3         │
│ 125 ┆ 2018-04-19 00:00:00 ┆ 1          │
│ 284 ┆ 2018-04-18 00:00:00 ┆ -14        │
│ 78  ┆ 2018-04-19 00:00:00 ┆ -5         │
│ …   ┆ …                   ┆ …          │
│ 715 ┆ 2018-04-21 00:00:00 ┆ 6          │
│ 109 ┆ 2018-04-21 00:00:00 ┆ 4          │
│ 407 ┆ 2018-04-20 00:00:00 ┆ -5         │
│ 659 ┆ 2018-04-19 00:00:00 ┆ 5          │
│ 204 ┆ 2018-04-23 00:00:00 ┆ 0          │
└─────┴─────────────────────┴────────────┘
